In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat
from torchvision import transforms

print(torch.__version__)        
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime
from lark.config import Config
from lark.learner import Learner
from lark.ops import Sig2Spec, MixedSig2Spec
from lark.data import *

1.8.1
0.8.0a0+e4e171a


In [3]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [4]:
# get list of models
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to /home/koen/.cache/torch/hub/master.zip


['resnest101',
 'resnest200',
 'resnest269',
 'resnest50',
 'resnest50_fast_1s1x64d',
 'resnest50_fast_1s2x40d',
 'resnest50_fast_1s4x24d',
 'resnest50_fast_2s1x64d',
 'resnest50_fast_2s2x40d',
 'resnest50_fast_4s1x64d',
 'resnest50_fast_4s2x40d']

In [5]:
cfg = Config(
    n_workers=12,
    noise_nsr_dbs = [30, 20, 10, 3],
    sites=None, 
    use_neptune=True, 
    log_batch_metrics=False,
    n_epochs=15, 
    bs=32,
    lr=1e-3, 
    model='resnest50',
    scheduler='torch.optim.lr_scheduler.CosineAnnealingLR'
)

In [6]:
cfg.as_dict()

{'sites': None,
 'data_dir': 'data/birdclef-2021',
 'checkpoint_dir': 'checkpoints',
 'bs': 32,
 'n_workers': 12,
 'train_duration': 5,
 'valid_duration': 5,
 'seed': 231,
 'use_pink_noise': 0.5,
 'use_recorded_noise': 0.5,
 'use_secondary_labels': False,
 'noise_nsr_dbs': [30, 20, 10, 3],
 'noise_dir': 'data/noise/BirdVox-DCASE-20k/wav-32k',
 'use_overlays': True,
 'max_overlays': 5,
 'overlay_weights': [0.71986223,
  0.21010333,
  0.06314581,
  0.00574053,
  0.00114811],
 'overlay_snr_dbs': [20, 10, 3],
 'apply_filter': 0.6,
 'use_neptune': True,
 'log_batch_metrics': False,
 'sr': 32000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'f1_threshold': 0.5,
 'lr': 0.001,
 'n_epochs': 15,
 'model': 'resnest50',
 'optimizer': 'torch.optim.Adam',
 'loss_fn': 'torch.nn.BCEWithLogitsLoss',
 'scheduler': 'torch.optim.lr_scheduler.CosineAnnealingLR',
 'labels': ['acafly',
  'acowoo',
  'aldfly',
  'ameavo',
  'amecro',
  'amegfi',
  '

In [7]:
cfg.training_dataset_size

397000

In [8]:
prep = MixedSig2Spec(cfg)
main_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
posp = torch.nn.Sequential(
    torch.nn.Linear(in_features=2048, out_features=1024, bias=True),
    torch.nn.Dropout(p=0.2),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=1024, out_features=512, bias=True),
    torch.nn.Dropout(p=0.2),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=512, out_features=len(cfg.labels), bias=True),
)
main_model.fc = posp
model = torch.nn.Sequential(prep, main_model)
model = model.cuda()


Using cache found in /home/koen/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [10]:
lrn = Learner("resnest50-with-filters-full", cfg, model)

resnest50-with-filters-full-20210512-222102


In [11]:
# x, y = next(iter(lrn.tdl))
# model(x.cuda())

In [12]:
lrn.learn()

psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-128


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/12407 [00:00<?, ?it/s]

Unexpected error in ping thread.
Traceback (most recent call last):
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/bravado/http_future.py", line 335, in unmarshal_response
    incoming_response.swagger_result = unmarshal_response_inner(  # type: ignore
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/bravado/http_future.py", line 370, in unmarshal_response_inner
    response_spec = get_response_spec(status_code=response.status_code, op=op)
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/bravado_core/response.py", line 157, in get_response_spec
    raise MatchingResponseNotFound(
bravado_core.exception.MatchingResponseNotFound: Response specification matching http status_code 408 not found for operation Operation(ping). Either add a response specification for the status_code or use a `default` response.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/

...

  0%|          | 0/75 [00:00<?, ?it/s]

2021-05-14 03:01:22 epoch:  14 train loss: 0.016255 train f1: 0.297454 valid loss: 0.008295 valid f1: 0.070229


  0%|          | 0/12407 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

2021-05-14 05:04:18 epoch:  15 train loss: 0.016271 train f1: 0.297751 valid loss: 0.008288 valid f1: 0.082426


In [13]:
lrn.scheduler.state_dict()

[autoreload of lark.config failed: Traceback (most recent call last):
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/koen/apps/anaconda3/envs/lark/lib/python3.8/site-packages/IPython/extensions/autoreload.p

{'T_max': 10,
 'eta_min': 1e-05,
 'base_lrs': [0.001],
 'last_epoch': 15,
 '_step_count': 16,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0005050000000000023]}

In [14]:
lrn.evaluate()

  0%|          | 0/75 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,1183,0,951617,0,0.002480
1,0.1,212,950394,1223,971,0.161956
2,0.2,128,951219,398,1055,0.149795
3,0.3,90,951441,176,1093,0.124224
4,0.4,65,951528,89,1118,0.097233
5,0.5,53,951567,50,1130,0.082426
6,0.6,33,951578,39,1150,0.052590
7,0.7,23,951593,24,1160,0.037398
8,0.8,13,951604,13,1170,0.021505
9,0.9,10,951612,5,1173,0.016694


In [15]:
lrn.load_checkpoint('best')

{'epoch': 11,
 'valid_loss': 0.00823017676981787,
 'valid_score': 0.07902735471725464}

In [16]:
lrn.evaluate()

  0%|          | 0/75 [00:00<?, ?it/s]

,thresh,tp,tn,fp,fn,f1
0,0.0,1183,0,951617,0,0.002480
1,0.1,219,950367,1250,964,0.165158
2,0.2,122,951229,388,1061,0.144123
3,0.3,81,951398,219,1102,0.109238
4,0.4,66,951493,124,1117,0.096140
5,0.5,52,951536,81,1131,0.079027
6,0.6,38,951562,55,1145,0.059561
7,0.7,23,951576,41,1160,0.036889
8,0.8,13,951592,25,1170,0.021294
9,0.9,7,951607,10,1176,0.011667
